In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_cpu_stationary_1d"
config.target = "jmlr"
config.debug_mode = True

plt.rcParams.update(config.tueplots_bundle())

## Problem Definition

In [ ]:
import cpu

domain = cpu.domain_1D

diffop = cpu.diffop_1D

In [ ]:
%matplotlib inline

import matplotlib.axes
import matplotlib.ticker

from probnum.typing import ArrayLike

from linpde_gp.typing import RandomProcessLike, RandomVariableLike


plt_grid = domain.uniform_grid(100)


def plot_belief(
    ax: matplotlib.axes.Axes,
    *,
    u: pn.randprocs.GaussianProcess,
    conditioned_on: list[str] = [],
    X_pde: ArrayLike | None = None,
    q_dot_A: RandomVariableLike | None = None,
    X_dts: ArrayLike | None = None,
    y_dts: ArrayLike | None = None,
    noise_dts: RandomVariableLike | None = None,
    solution: RandomProcessLike | None = None,
):
    # Solution Belief
    cond_events_str = ", ".join(_build_cond_events_str(conditioned_on))

    u.plot(
        ax,
        plt_grid,
        num_samples=10,
        rng=np.random.default_rng(24),
        color="C0",
        label=(
            fr"$\mathrm{{u}} \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else r"$\mathrm{u}$"
        ),
    )
    
    # True Solution
    if solution is not None:
        ax.plot(
            plt_grid,
            solution(plt_grid),
            color="C1",
            label="$u^*$",
        )

    for key in conditioned_on:
        # PDE Observations
        if key == "pde":
            X_pde = np.asarray(X_pde)

            for i, x in enumerate(X_pde):
                ax.axvline(
                    x,
                    color="C3",
                    alpha=0.1,
                    linestyle="--",
                    label=r"$\bm{X}_{\mathrm{PDE}}$" if i == 0 else None,
                )
        # Neumann Boundary Conditions
        elif key == "nbc":
            X_bc = np.asarray(domain.boundary)
            q_dot_A = pn.randvars.asrandvar(q_dot_A)

            slopes = [-q_dot_A[0] / cpu.kappa, q_dot_A[1] / cpu.kappa]

            linpde_gp.utils.plotting.plot_local_taylor_processes(
                ax,
                xs=X_bc,
                coeffs_xs=[
                    pn.randvars.Normal(
                        np.array([u.mean(x), slope.mean]),
                        np.diag([0.0, slope.var]),
                    )
                    for x, slope in zip(domain, slopes)
                ],
                radius=0.3,
                color="C5",
                rel_fill_alpha=0.25,
                label=r"$\dot{\mathrm{q}}_A(\bm{X}_{\mathrm{NBC}})" + (
                    r"\mid \mathrm{STAT}$"
                    if "stat" in conditioned_on
                    else "$"
                ),
            )
        # Measurements
        elif key == "dts":
            X_dts = np.asarray(X_dts)
            y_dts = np.asarray(y_dts)
            noise_dts = pn.asrandvar(noise_dts)

            ax.errorbar(
                X_dts,
                y_dts + noise_dts.mean,
                yerr=1.96 * noise_dts.std,
                fmt="+",
                capsize=2,
                color="C4",
                label=r"$(\bm{X}_{\mathrm{DTS}}, \bm{y}_{\mathrm{DTS}})$",
            )

    cpu.adjust_xaxis(ax)
    cpu.adjust_tempaxis(ax.yaxis)

    # ax.set_ylabel(
    #     r"Temperature (\unit{\degreeCelsius})",
    #     ha="left",
    #     y=1,
    #     rotation=0,
    #     labelpad=0,
    # )

    ax.legend()


def plot_pred_belief(
    ax,
    u: pn.randprocs.GaussianProcess | None = None,
    q_dot_V: RandomProcessLike | None = None,
    conditioned_on: list[str] = [],
    X_pde: ArrayLike | None = None,
):
    # Differential Operator Image Belief
    if u is not None:
        cond_events_str = ", ".join(_build_cond_events_str(conditioned_on))

        diffop(u).plot(
            ax,
            plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
                label=(
                fr"$-\kappa \Delta \mathrm{{u}} \mid {cond_events_str}$"
                if len(cond_events_str) > 0
                else r"$-\kappa \Delta \mathrm{u}$"
            ),
        )
    
    # True Right-Hand Side
    if q_dot_V is not None:
        q_dot_V_label = r"\dot{" + (
            r"\mathrm{q}" if isinstance(q_dot_V, pn.randprocs.RandomProcess) else "q"
        ) + r"}_V"

        q_dot_V = linpde_gp.randprocs.asrandproc(q_dot_V)

        q_dot_V.plot(
            ax,
            plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C1",
            label="$" + q_dot_V_label + (
                r"\mid \mathrm{STAT}$"
                if "stat" in conditioned_on
                else "$"
            ),
        )
    
    # PDE Observations
    if "pde" in conditioned_on:
        X_pde = np.asarray(X_pde)
        q_dot_V_X_pde = q_dot_V(X_pde)

        ax.errorbar(
            X_pde,
            q_dot_V_X_pde.mean,
            yerr=1.96 * q_dot_V_X_pde.std,
            fmt="+",
            capsize=2,
            c="C3",
            label=(
                r"$(\bm{X}_{\mathrm{PDE}},"
                f"{q_dot_V_label}"
                r"(\bm{X}_{\mathrm{PDE}})$"
            ),
        )
    
    cpu.adjust_xaxis(ax)
    cpu.adjust_q_dot_V_axis(ax.yaxis)

    # ax.set_ylabel(
    #     r"Heat Flow (\unit{\watt\per\cubic\mm)}",
    #     ha="left",
    #     y=1,
    #     rotation=0,
    #     labelpad=0,
    # )

    ax.legend()


def _build_cond_events_str(
    conditioned_on: list[str]
) -> str:
    for key in conditioned_on:
        match key:
            case "pde":
                yield r"\mathrm{PDE}"
            case "dbc":
                yield r"\mathrm{DBC}"
            case "nbc":
                yield r"\mathrm{NBC}"
            case "dts":
                yield r"\mathrm{DTS}"
            case "stat":
                if "pde" in conditioned_on:
                    yield r"\mathrm{STAT}"
                else:
                    continue
            case _:
                raise ValueError(f"Unknown event '{key}'")

## Simplified Model with Dirichlet Boundaries

### Prior

In [ ]:
u = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=2.5,
        lengthscales=cpu.width,
    ),
)

q_dot_V = cpu.q_dot_V_src_1D + linpde_gp.functions.Constant(
    input_shape=(),
    value=-cpu.TDP / cpu.V,
)

y_dbc = cpu.u_X_dbc_1D

In [ ]:
plot_belief(
    ax=plt.gca(),
    u=u,
)

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    u=u,
    q_dot_V=q_dot_V,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u,
            q_dot_V=q_dot_V,
        )

experiment_utils.savefig("01_simplified_dbc_00_prior")

### Observations

In [ ]:
N_pde = 15

X_pde = domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

### Posterior

#### Conditioning on the PDE

In [ ]:
u_cond_pde = u.condition_on_observations(
    Y=q_dot_V(X_pde),
    X=X_pde,
    L=diffop,
)

In [ ]:
plot_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    q_dot_V=q_dot_V,
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde,
            q_dot_V=q_dot_V,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("01_simplified_dbc_01_cond_pde")

#### Conditioning on the Dirichlet Boundary Conditions

In [ ]:
X_dbc = np.asarray(domain.boundary)

u_cond_pde_bc = u_cond_pde.condition_on_observations(
    Y=y_dbc,
    X=X_dbc,
)

In [ ]:
def plot_dbc(ax: matplotlib.axes.Axes, u_X_dbc: RandomVariableLike):
    X_dbc = np.asarray(domain.boundary)
    u_X_dbc = pn.randvars.asrandvar(u_X_dbc)

    ax.errorbar(
        X_dbc,
        u_X_dbc.mean,
        yerr=1.96 * u_X_dbc.std,
        fmt="+",
        capsize=2,
        color="C2",
        label=r"$g$",
    )

In [ ]:
plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "dbc"],
    X_pde=X_pde,
)

plot_dbc(plt.gca(), u_X_dbc=y_dbc)

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    q_dot_V=q_dot_V,
    conditioned_on=["pde", "dbc"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde_bc,
            conditioned_on=["pde", "dbc"],
            X_pde=X_pde,
        )

        plot_dbc(ax[0], u_X_dbc=y_dbc)

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_bc,
            q_dot_V=q_dot_V,
            conditioned_on=["pde", "dbc"],
            X_pde=X_pde,
        )
    
experiment_utils.savefig("01_simplified_dbc_02_cond_pde_dbc")

## Simplified Model with Uncertain Neumann Boundary Conditions and Interior Measurements

### Prior

In [ ]:
u = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=2.5,
        lengthscales=cpu.width,
    ),
)

q_dot_V = cpu.q_dot_V_1D

q_dot_A = pn.randvars.Normal(
    mean=cpu.q_dot_A_1D,
    cov=np.diag(np.abs(cpu.q_dot_A_1D) ** 2),
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            ax=plt.gca(),
            u=u,
        )

experiment_utils.savefig("02_simplified_nbc_00_prior_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u,
            q_dot_V=q_dot_V,
        )

experiment_utils.savefig("02_simplified_nbc_00_prior_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.44)):
    fig, ax = plt.subplots(nrows=2, sharex=True)

    # CPU Schematic
    cpu.plot_schematic(ax[0])

    ax[0].imshow(
        cpu.q_dot_V_2D(
            np.stack(
                np.meshgrid(
                    np.linspace(*cpu.domain_2D[0], 50),
                    np.linspace(*cpu.domain_2D[1], 50)
                ),
                axis=-1,
            )
        ),
        cmap="coolwarm",
        norm=matplotlib.colors.TwoSlopeNorm(0.0),
        extent=[0.0, cpu.width, 0.0, cpu.height],
    )

    ax[0].plot([0.0, cpu.width], 2 * [cpu.core_centers_ys[1]], c="C1")

    # PDE RHS
    plt_grid = np.linspace(*domain, 100)

    cpu.q_dot_V_1D.plot(
        ax[1],
        plt_grid,
        c="C1",
        label=r"$\dot{q}_V$"
    )

    cpu.adjust_xaxis(ax[1])
    cpu.adjust_q_dot_V_axis(ax[1].yaxis)

    ax[1].legend(loc="lower center")

experiment_utils.savefig("01_simplified_nbc_00_geometry_rhs")

### Observations

In [ ]:
N_pde = 15

X_pde = domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

In [ ]:
X_dts = cpu.X_dts_1D
y_dts = cpu.y_dts_1D
noise_dts = cpu.noise_dts_1D

### Posterior

#### Conditioning on the PDE

In [ ]:
u_cond_pde = u.condition_on_observations(
    Y=q_dot_V(X_pde),
    X=X_pde,
    L=diffop,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde,
            q_dot_V=q_dot_V,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde,
            q_dot_V=q_dot_V,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde")

#### Conditioning on Uncertain Neumann Boundary Conditions

According to Fourier's law, the outward heat flux through the surface of the CPU is
given by $q(x) = \langle n(x), - k \nabla u(x) \rangle = - k \nabla_{n(x)} u(x)$.

In [ ]:
u_cond_pde_nbc_left = u_cond_pde.condition_on_observations(
    X=domain[0],
    Y=0.0,
    L=-cpu.kappa * linpde_gp.linfuncops.diffops.DirectionalDerivative(1.0),
    b=-q_dot_A[0],
)

u_cond_pde_nbc = u_cond_pde_nbc_left.condition_on_observations(
    X=domain[1],
    Y=0.0,
    L=-cpu.kappa * linpde_gp.linfuncops.diffops.DirectionalDerivative(-1.0),
    b=-q_dot_A[1],
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
            q_dot_A=q_dot_A,
        )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc,
            q_dot_V=q_dot_V,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde_nbc,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
            q_dot_A=q_dot_A,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_nbc,
            q_dot_V=q_dot_V,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc")

#### Conditioning on Noisy Interior Measurements

In [ ]:
u_cond_pde_nbc_dts = u_cond_pde_nbc.condition_on_observations(
    Y=y_dts,
    X=X_dts,
    b=noise_dts,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc_dts,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
            q_dot_A=q_dot_A,
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc_dts,
            q_dot_V=q_dot_V,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde_nbc_dts,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
            q_dot_A=q_dot_A,
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_nbc_dts,
            q_dot_V=q_dot_V,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts")

## Full Model

\begin{align*}
    \mathrm{u} & \sim \mathcal{GP}(m_u, k_u) \\
    \dot{\mathrm{q}}_V & \sim \mathcal{GP}(m_f, k_f) \\
    \dot{\bm{\mathrm{q}}}_A & \sim \mathcal{N}(\bm{\mu}_g, \bm{\Sigma}_g) \\
    \bm{\epsilon}_{\text{DTS}} & \sim \mathcal{N}(0, \sigma_{\text{DTS}}^2 \bm{I})
\end{align*}

\begin{align*}
    w_\text{CPU} \cdot \int_{\mathbb{D}} \dot{\mathrm{q}}_V(x) \mathrm{d}x + h_\text{CPU} \cdot \left( \dot{\bm{\mathrm{q}}}_{A, 1} + \dot{\bm{\mathrm{q}}}_{A, 2} \right) & = 0 \\
    -\kappa \Delta \mathrm{u}(\bm{X}_{\text{PDE}}) - \dot{\mathrm{q}}_V(\bm{X}_{\text{PDE}}) & = 0 \\
    -\kappa \nabla_{\nu(\bm{X}_{\text{NBC}})} \mathrm{u}(\bm{X}_{\text{NBC}}) - \dot{\bm{\mathrm{q}}}_{A} & = 0 \\
    \mathrm{u}(\bm{X}_{\text{DTS}}) + \bm{\epsilon}_{\text{DTS}} & = \bm{y}_{\text{DTS}}
\end{align*}

### Prior

In [ ]:
u = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=2.5,
        lengthscales=cpu.width,
    ),
)

In [ ]:
q_dot_V = pn.randprocs.GaussianProcess(
    mean=cpu.q_dot_V_1D,
    cov=0.9 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=3.5,
        lengthscales=cpu.width / 2,
    ),
)

In [ ]:
q_dot_A = pn.randvars.Normal(
    mean=cpu.q_dot_A_1D,
    cov=np.diag(np.abs(cpu.q_dot_A_1D) ** 2),
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=u,
    q_dot_V=q_dot_V,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            q_dot_V=q_dot_V,
        )

experiment_utils.savefig("03_full_00_q_dot_V_prior")

### Observations

In [ ]:
N_pde = 20

X_pde = domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

In [ ]:
X_dts = cpu.X_dts_1D
y_dts = cpu.y_dts_1D
noise_dts = cpu.noise_dts_1D

### Posterior

#### Conditioning on (Thermal) Stationarity

In [ ]:
L_stat = cpu.height * linpde_gp.linfunctls.LebesgueIntegral(input_domain=domain)
A_stat = np.array([cpu.height, cpu.height])

q_dot_V_cond_stat = q_dot_V.condition_on_observations(
    Y=0.0,
    L=L_stat,
    b=A_stat @ q_dot_A,
)

q_dot_A_cond_stat = q_dot_A.condition_on_observations(
    observations=0.0,
    noise=L_stat(q_dot_V),
    transform=A_stat,
)

class CrossCovariance_q_dot_V_A_cond_stat(
    linpde_gp.randprocs.crosscov.ProcessVectorCrossCovariance
):
    def __init__(self):
        self._kLa = L_stat(q_dot_V.cov, argnum=1)
        self._A_Sigma = A_stat @ q_dot_A.cov

        super().__init__(
            randproc_input_shape=q_dot_V.input_shape,
            randproc_output_shape=q_dot_V.output_shape,
            randvar_shape=q_dot_A.shape,
            reverse=False
        )
    
    def _evaluate(self, x: np.ndarray) -> np.ndarray:
        return -self._kLa(x)[:, None] * self._A_Sigma[None, :] / q_dot_V_cond_stat.gram[0, 0]
    
    def _evaluate_jax(self, x):
        pass

q_dot_V_A_cond_stat_crosscov = CrossCovariance_q_dot_V_A_cond_stat()

In [ ]:
q_dot_A_cond_stat.cov

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=u,
    q_dot_V=q_dot_V_cond_stat,
    conditioned_on=["stat"],
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            q_dot_V=q_dot_V_cond_stat,
            conditioned_on=["stat"],
        )

experiment_utils.savefig("03_full_01_q_dot_V_cond_stat")

#### Conditioning on the PDE with Uncertain RHS

In [ ]:
u_cond_pde_stat_uncorr = u.condition_on_observations(
    Y=np.zeros_like(X_pde),
    X=X_pde,
    L=diffop,
    b=-q_dot_V_cond_stat(X_pde),
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_pde_stat_uncorr,
    conditioned_on=["pde", "stat"],
    X_pde=X_pde,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=u_cond_pde_stat_uncorr,
    q_dot_V=q_dot_V_cond_stat,
    conditioned_on=["pde", "stat"],
    X_pde=X_pde,
)

#### Conditioning on Uncertain Neumann Boundary Conditions

In [ ]:
u_cond_pde_nbc_left_stat_uncorr = u_cond_pde_stat_uncorr.condition_on_observations(
    X=domain[0],
    Y=0.0,
    L=-cpu.kappa * linpde_gp.linfuncops.diffops.DirectionalDerivative(1.0),
    b=-q_dot_A_cond_stat[0],
)

u_cond_pde_nbc_stat_uncorr = u_cond_pde_nbc_left_stat_uncorr.condition_on_observations(
    X=domain[1],
    Y=0.0,
    L=-cpu.kappa * linpde_gp.linfuncops.diffops.DirectionalDerivative(-1.0),
    b=-q_dot_A_cond_stat[1],
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_pde_nbc_stat_uncorr,
    conditioned_on=["pde", "nbc", "stat"],
    X_pde=X_pde,
    q_dot_A=q_dot_A_cond_stat,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=u_cond_pde_nbc_stat_uncorr,
    q_dot_V=q_dot_V_cond_stat,
    conditioned_on=["pde", "nbc", "stat"],
    X_pde=X_pde,
)

### Modify Gram matrix to account for correlations

In [ ]:
gram_blocks = u_cond_pde_nbc_stat_uncorr._gram_blocks
q_dot_V_A_cond_stat_crosscov_X_pde = q_dot_V_A_cond_stat_crosscov(X_pde)

In [ ]:
gram_blocks_corr = (
    (gram_blocks[0][0],),
    (gram_blocks[1][0] + q_dot_V_A_cond_stat_crosscov_X_pde[:, 0].T, gram_blocks[1][1]),
    (gram_blocks[2][0] + q_dot_V_A_cond_stat_crosscov_X_pde[:, 1].T, gram_blocks[2][1] + q_dot_A_cond_stat.cov[1, 1], gram_blocks[2][2])
)

In [ ]:
u_cond_pde_nbc_stat = linpde_gp.randprocs.ConditionalGaussianProcess(
    prior=u_cond_pde_nbc_stat_uncorr._prior,
    Ys=u_cond_pde_nbc_stat_uncorr._Ys,
    Ls=u_cond_pde_nbc_stat_uncorr._Ls,
    bs=u_cond_pde_nbc_stat_uncorr._bs,
    kLas=u_cond_pde_nbc_stat_uncorr._kLas,
    gram_blocks=gram_blocks_corr,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_pde_nbc_stat,
    conditioned_on=["pde", "nbc", "stat"],
    X_pde=X_pde,
    q_dot_A=q_dot_A_cond_stat,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=u_cond_pde_nbc_stat,
    q_dot_V=q_dot_V_cond_stat,
    conditioned_on=["pde", "nbc", "stat"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde_nbc_stat,
            conditioned_on=["pde", "nbc", "stat"],
            X_pde=X_pde,
            q_dot_A=q_dot_A_cond_stat,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_nbc_stat,
            q_dot_V=q_dot_V_cond_stat,
            conditioned_on=["pde", "nbc", "stat"],
            X_pde=X_pde,
        )

experiment_utils.savefig("03_full_02_cond_pde_nbc_stat")

### Measurements

In [ ]:
u_cond_pde_nbc_stat_dts = u_cond_pde_nbc_stat.condition_on_observations(
    Y=y_dts,
    X=X_dts,
    b=noise_dts,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            plt.gca(),
            u=u_cond_pde_nbc_stat_dts,
            conditioned_on=["pde", "nbc", "stat", "dts"],
            X_pde=X_pde,
            q_dot_A=q_dot_A_cond_stat,
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

experiment_utils.savefig("03_full_03_cond_pde_nbc_stat_dts_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            plt.gca(),
            u=u_cond_pde_nbc_stat_dts,
            q_dot_V=q_dot_V_cond_stat,
            conditioned_on=["pde", "nbc", "stat", "dts"],
            X_pde=X_pde,
        )

experiment_utils.savefig("03_full_03_cond_pde_nbc_stat_dts_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plot_belief(
            ax=ax[0],
            u=u_cond_pde_nbc_stat_dts,
            conditioned_on=["pde", "nbc", "stat", "dts"],
            X_pde=X_pde,
            q_dot_A=q_dot_A_cond_stat,
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

        plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_nbc_stat_dts,
            q_dot_V=q_dot_V_cond_stat,
            conditioned_on=["pde", "nbc", "stat", "dts"],
            X_pde=X_pde,
        )

experiment_utils.savefig("03_full_03_cond_pde_nbc_stat_dts")